In [ ]:
import cupy as cp
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm
import imageio
import gc
import pandas as pd
import cartopy.crs as ccrs


class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2

        self.stars_img = np.array(Image.open("../data/Background_1.jpg").convert("RGB"))
        self.stars_height, self.stars_width = self.stars_img.shape[:2]

        self.image_size_x = self.stars_width
        self.image_size_y = self.stars_height
        
        self.r_start = 250 * self.rs
        self.r_max = self.r_start * 1.1

        self.b_max_x = 50 * self.rs / self.M
        self.b_max_y = self.b_max_x / self.image_size_x * self.image_size_y
        self.b_max = np.sqrt(self.b_max_x**2 + self.b_max_y**2)
        
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.from_numpy(phi0).to(dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=m-1
        with tqdm(total=N, desc="Calculando trayectorias", unit="Tr") as pbar:
          while t < tf and active.any():
            #while error>
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)

            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            n +=1
            if n==m:
              ys.append(y.cpu())
              n=0
              pbar.update(1)
            t += h

        return torch.stack(ys)
    
    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx
    
    def mollweide(self, gamma_deg, x1, y1, z1, lon0=np.pi/2):
        #x_rot, y_rot, z_rot =  [x1, y1, z1] @ self.matriz_rotacion(gamma_deg=gamma_deg) 
        x_rot = x1
        y_rot = z1
        z_rot = y1

        lat = np.arcsin(z_rot)
        lon = ((np.arctan2(y_rot, x_rot))) % (2 * np.pi)

        lambda_ = lon
        lambda_0 = lon0

        # Si está en los polos, evitar iteración y asignar theta directamente
        if abs(abs(lat) - np.pi / 2) < 1e-5:
            lat = np.copysign(np.pi / 2, lat)
            theta = lat
        else:
            # Newton-Raphson para resolver: 2θ + sin(2θ) = π sin(φ)
            theta = lat  # inicialización
            delta = 1
            while np.abs(delta)>1e-6:
                numerator = 2 * theta + np.sin(2 * theta) - np.pi * np.sin(lat)
                denominator = 2 + 2 * np.cos(2 * theta) 
                delta = numerator / denominator
                theta -= delta

        # Ecuaciones de la proyección Mollweide:
        x = (1.0 / np.pi ) * ((lambda_ - lambda_0)) * np.cos(theta) 
        y = np.sin(theta)

        return x , y

    def pixel(self, gamma_deg, x1, y1, z1):

        x, y = self.mollweide(gamma_deg, x1, y1, z1)

        i = int(x * self.stars_width // 2 + self.stars_width//2) % self.stars_width
        j = int(y * self.stars_height // 2 + self.stars_height//2) % self.stars_height

        return i, j
    
    def color(self, gamma_deg, x1, y1, z1):

        x_pix, y_pix = self.pixel(gamma_deg, x1, y1, z1)
        colors = self.stars_img[y_pix, x_pix]

        return  tuple(int(c) for c in colors)
    
    def render(self, N_x=100, N_y=100, gamma_deg=20, ti=0, tf=2000):
        ancho, alto = self.stars_width//4, self.stars_height//4

        xs = np.linspace(0, 1, ancho//2)
        ys = np.linspace(0, 1, alto//2)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = np.sqrt((self.b_max_x*xv.flatten())**2 + (self.b_max_y*yv.flatten())**2)
        r0= self.r_start

        print(2 * np.arcsin(self.b_max/r0)/np.pi*180)
        N=500

        sol_batch = self.solve_batch(b_vals, phi0= np.arcsin(b_vals/r0), r0=r0, tau_span=(ti, tf), h=0.05, N=N)

        phi_vals = sol_batch[:, :, 1].numpy()
        r_vals = sol_batch[:, :, 0].numpy()

        print(r_vals[-1,100])

        del sol_batch #, r_vals
        gc.collect()
        
        n=0

        gamma_deg_vals=np.linspace(0,180, 1)
        with tqdm(total=len(gamma_deg_vals), desc="Renderizando", unit="Imágenes") as pbar:
          with imageio.get_writer("../Multimedia/Agujero_negro/Fonfo_estrellado.mp4", fps=10) as writer1, \
            imageio.get_writer("../Multimedia/Agujero_negro/Fonfo_estrellado_2.mp4", fps=10) as writer2 :
            for gamma_deg in gamma_deg_vals:
                print(gamma_deg)
                imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
                imagen2 = Image.new('RGB', (ancho, alto), color=(0,0,0))
                pixeles = imagen.load()
                pixeles2= imagen2.load()
                idx=0

                for i in range(ancho//2):
                    for j in range(alto//2):
                        b = np.sqrt((self.b_max_x*xv[j,i])**2 + (self.b_max_y*yv[j,i])**2)

                        if b <= 3 * np.sqrt(3) / 2 * self.rs:
                            pixeles[ancho//2+i, alto//2 +j] = (0,0,0)
                            pixeles[ancho//2-i, alto//2 +j] = (0,0,0)
                            pixeles[ancho//2+i, alto//2 -j] = (0,0,0)
                            pixeles[ancho//2-i, alto//2 -j] = (0,0,0)

                            phi0=np.arcsin(b/r0)

                            theta = np.arctan2(self.b_max_y*yv[j,i], self.b_max_x*xv[j,i])

                            x_traj =  np.cos(theta) * np.sin(phi0)
                            y_traj =  np.sin(theta) * np.sin(phi0)
                            z_traj =  -np.cos(phi0)

                            pixeles2[ancho//2+i, alto//2 +j]=self.color(gamma_deg, x_traj, y_traj, -z_traj)
                            pixeles2[ancho//2-i, alto//2 +j]=self.color(gamma_deg, -x_traj, y_traj, -z_traj)
                            pixeles2[ancho//2+i, alto//2 -j]=self.color(gamma_deg, x_traj, -y_traj, -z_traj)
                            pixeles2[ancho//2-i, alto//2 -j]=self.color(gamma_deg, -x_traj, -y_traj, -z_traj)

                        else:
                            phi0=np.arcsin(b/r0)

                            phi_traj = phi_vals[-1, idx] % (2 * np.pi) 

                            if r_vals[-1,idx]<self.r_max:
                                print(r_vals[-1,idx])
                                n+=1

                            theta = np.arctan2(self.b_max_y*yv[j,i], self.b_max_x*xv[j,i])

                            x_traj =  np.cos(theta) * np.sin(phi_traj)
                            y_traj =  np.sin(theta) * np.sin(phi_traj)
                            z_traj =  -np.cos(phi_traj)

                            pixeles[ancho//2+i, alto//2 +j] = self.color(gamma_deg, x_traj, y_traj, z_traj)
                            pixeles[ancho//2-i, alto//2 +j] = self.color(gamma_deg, -x_traj, y_traj, z_traj)
                            pixeles[ancho//2+i, alto//2 -j] = self.color(gamma_deg, x_traj, -y_traj, z_traj)
                            pixeles[ancho//2-i, alto//2 -j] = self.color(gamma_deg, -x_traj, -y_traj, z_traj)

                            x_traj =  np.cos(theta) * np.sin(phi0)
                            y_traj =  np.sin(theta) * np.sin(phi0)
                            z_traj =  -np.cos(phi0)

                            pixeles2[ancho//2+i, alto//2 +j]=self.color(gamma_deg, x_traj, y_traj, -z_traj)
                            pixeles2[ancho//2-i, alto//2 +j]=self.color(gamma_deg, -x_traj, y_traj, -z_traj)
                            pixeles2[ancho//2+i, alto//2 -j]=self.color(gamma_deg, x_traj, -y_traj, -z_traj)
                            pixeles2[ancho//2-i, alto//2 -j]=self.color(gamma_deg, -x_traj, -y_traj, -z_traj)

                        idx += 1
                print(n)
                pbar.update(1)
                frame = np.array(imagen)
                writer1.append_data(frame)
                combined_image = Image.new('RGB', (2 * ancho, alto))
                combined_image.paste(imagen, (0, 0))
                combined_image.paste(imagen2, (ancho, 0))
                frame2 = np.array(combined_image)
                writer2.append_data(frame2)

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
#rt_batch.render(N_x=224, N_y=112)
rt_batch.render()
#rt_batch.render(N_x=1024, N_y=512)

25.84193276316713


Calculando trayectorias: 501Tr [07:50,  1.06Tr/s]                         


1499.2637


Renderizando:   0%|          | 0/1 [00:00<?, ?Imágenes/s]

0.0
1.9880219
1.984944
1.9975007
1.9896673
1.9856309
1.9916751
1.9967942
1.9904945
1.9970431
1.9618751
1.9850599
1.9949648
1.9766066
1.98815
1.9573462
1.9622412
1.9889735
1.961096
1.9663044
1.9512218
1.9860878
1.9852812
1.9682518
1.9768872
1.951416
1.9585663
1.9656616
1.9506503
1.9900507
1.9588242
1.963703
1.9600804
1.9603318
1.9825188
1.996458
1.9579581
1.992346
1.9806391
1.9694669
1.9905868
1.9833126
1.9504255
1.9970915
1.9751335
1.9911126
1.9517177
1.9745203
1.9732083
1.9820493
1.9734657
1.9824569
1.9587345
1.9860138
1.9804869
1.993793
1.9766032
1.9797838
1.953808
1.9521513
1.9776354
1.9826051
1.9718448
1.9512136
1.9845597
1.9842032
1.9697602
1.9936984
1.9565616
1.9961876
1.9728433
1.9821246
1.9595618
1.9560456
1.9720083
1.957442
1.9646134
1.995271
1.9501032
1.9847164
1.9499756
1.9534028
1.9533279
1.9631352
1.9973658
1.9901977
1.9609408
1.9874688
1.9819376
1.9714191
1.9811188
1.9595137
1.9577655
1.9768056
1.9665922
1.9795566
1.9667127
1.9816253
1.9765825
1.9561301
1.9798055
1.955895

Renderizando: 100%|██████████| 1/1 [00:15<00:00, 15.63s/Imágenes]IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (810, 405) to (816, 416) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1620, 405) to (1632, 416) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


226


Renderizando: 100%|██████████| 1/1 [00:15<00:00, 15.89s/Imágenes]


In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from scipy import optimize
from scipy.special import ellipkinc  # ✅ usamos ellipkinc desde scipy

class GPU_SchwarzschildRayTracer:
    def __init__(self, beta=0.0, bh_mass=1.0):
        self.stars_img = np.array(Image.open("../data/Background_1.jpg").convert("RGB"))
        self.stars_height, self.stars_width = self.stars_img.shape[:2]

        self.image_size_x = self.stars_width
        self.image_size_y = self.stars_height

        self.M = bh_mass
        self.rs = 2 * bh_mass

        self.r_start = 100 * self.rs
        self.r_max = self.r_start * 1.1

        self.b_max_x = 20 * self.rs / self.M
        self.b_max_y = self.b_max_x / self.image_size_x * self.image_size_y
        self.b_max = np.sqrt(self.b_max_x**2 + self.b_max_y**2)

        self.beta = beta

        self._precompute_rm_table()

    def _precompute_rm_table(self, N=1000):
        b_vals = np.linspace(self.rs * 1.0001, self.b_max, N)
        rm_vals = []

        for b in tqdm(b_vals, desc="Precomputando rm(b)", unit="val"):
            def rmin(R): return 1 / b**2 - 1 / R**2 + self.rs / R**3
            try:
                sol = optimize.root_scalar(rmin, bracket=[self.rs * 1.01, self.r_max], method='brentq')
                rm_vals.append(sol.root)
            except:
                rm_vals.append(self.rs)

        self.b_vals_cpu = b_vals
        self.rm_vals_cpu = np.array(rm_vals)
        self.rm_interp_gpu = cp.interp  # usamos directamente cp.interp en GPU

    def deflexion_gpu(self, rm):
        # Deflexión de rayos: parte en GPU
        s = cp.sqrt((rm - self.rs) * (rm + 3 * self.rs))
        m = (s - rm + 3 * self.rs) / (2 * s)
        arg = cp.sqrt(2 * s / (3 * rm - 3 * self.rs + s))
        arg = cp.clip(arg, -1.0, 1.0)
        varphi = cp.arcsin(arg)

        # Paso a CPU para calcular ellipkinc
        varphi_cpu = cp.asnumpy(varphi)
        m_cpu = cp.asnumpy(m)
        rm_cpu = cp.asnumpy(rm)
        s_cpu = cp.asnumpy(s)

        alpha_cpu = 4 * np.sqrt(rm_cpu / s_cpu) * ellipkinc(varphi_cpu, m_cpu)
        alpha_gpu = cp.array(alpha_cpu)

        return (alpha_gpu % (2 * cp.pi)) - cp.pi

    def mollweide(self, theta, phi):
        delta, alpha = self.spherical_to_equatorial(theta, phi)
        x = ((delta / np.pi) * np.cos(alpha) + 1) / 2  # [0,1]
        y = (np.sin(alpha) + 1) / 2  # [0,1]
        return x, y

    def spherical_to_equatorial(self, theta, phi):
        x = np.sin(theta) * np.cos(phi)
        y = np.sin(theta) * np.sin(phi)
        z = np.cos(theta)

        x_rot = x
        y_rot = z
        z_rot = -y

        delta = np.arcsin(z_rot)
        alpha = ((np.arctan2(y_rot, x_rot)) - self.beta) % (2 * np.pi)

        return delta, alpha

    def pixel(self, phi, theta_view, alpha):
        phi = (phi / 10 - np.pi / 20).astype(cp.int32)
        x, y = self.mollweide(phi - alpha, theta_view)

        i = (x * self.stars_width).astype(cp.int32) % self.stars_width
        j = (y * self.stars_height).astype(cp.int32) % self.stars_height

        return i, j

    def render(self, save_path="../Multimedia/Agujero_negro/Agujero_negro_fondo_estrellado_lenteado.png"):
        i = cp.arange(self.image_size_x)
        j = cp.arange(self.image_size_y)
        ii, jj = cp.meshgrid(i, j, indexing='ij')

        x = (ii - self.image_size_x / 2) / (self.image_size_x / 2) * self.b_max_x
        y = (jj - self.image_size_y / 2) / (self.image_size_y / 2) * self.b_max_y

        b = cp.sqrt(x**2 + y**2)
        theta_view = cp.arctan2(y, x)

        mask = b > self.rs
        b_valid = cp.where(mask, b, self.rs * (1 + 1e-5))

        b_vals_gpu = cp.asarray(self.b_vals_cpu)
        rm_vals_gpu = cp.asarray(self.rm_vals_cpu)
        rm = cp.interp(b_valid, b_vals_gpu, rm_vals_gpu)

        alpha = self.deflexion_gpu(rm)
        phi = b_valid / self.b_max * cp.pi

        x_pix, y_pix = self.pixel(phi, theta_view, alpha)

        # Samplear colores desde imagen CPU
        colors = self.stars_img[y_pix.get(), x_pix.get()]
        # Opcional: oscurecer centro
        # colors[~mask.get()] = [0, 0, 0]

        final_img = colors.reshape((self.image_size_x, self.image_size_y, 3))
        plt.imshow(final_img.astype(np.uint8))
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()

        print(f"Imagen guardada en: {save_path}")

# Ejecutar trazado de rayos
rt = GPU_SchwarzschildRayTracer(beta=0.0, bh_mass=1.0)
rt.render()


Precomputando rm(b): 100%|██████████| 1000/1000 [00:00<00:00, 43235.34val/s]
C:\Users\User\AppData\Local\Temp\ipykernel_13548\373324684.py:61: RuntimeWarning: divide by zero encountered in divide
  alpha_cpu = 4 * np.sqrt(rm_cpu / s_cpu) * ellipkinc(varphi_cpu, m_cpu)


Imagen guardada en: ../Multimedia/Agujero_negro_fondo_estrellado_lenteado.png


In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from scipy import optimize

from cupyx.scipy.special import ellipk


class GPU_SchwarzschildRayTracer:
    def __init__(self, beta=0.0, bh_mass=1.0):
        # Imagen de fondo
        self.stars_img = np.transpose(np.array(Image.open("../data/Background_1.jpg").convert("RGB")))
        self.stars_width , self.stars_height = self.stars_img.shape[:2]

        self.image_size_x = self.stars_width
        self.image_size_y = self.stars_height

        # Parámetros físicos
        self.M = bh_mass
        self.rs = 2 * bh_mass

        self.r_start = 100 * self.rs
        self.r_max = self.r_start * 1.1

        self.b_max_x = 20 * self.rs / self.M
        self.b_max_y = self.b_max_x / self.image_size_x * self.image_size_y
        self.b_max = np.sqrt(self.b_max_x**2 + self.b_max_y**2)

        self.beta = beta

        # Precálculo de rm(b) en CPU
        self._precompute_rm_table()

    def _precompute_rm_table(self, N=1000):
        b_vals = np.linspace(self.rs * 1.01, self.b_max, N)
        rm_vals = []

        for b in tqdm(b_vals, desc="Precomputando rm(b)", unit="val"):
            def rmin(R): return 1 / b**2 - 1 / R**2 + self.rs / R**3

            try:
                sol = optimize.root_scalar(rmin, bracket=[self.rs * 1.01, self.r_max], method='brentq')
                rm_vals.append(sol.root)
            except:
                rm_vals.append(self.rs)

        self.b_vals_cpu = b_vals
        self.rm_vals_cpu = np.array(rm_vals)
        self.rm_interp_gpu = cp.interp  # usaremos cp.interp directamente en GPU

    def deflexion_gpu(self, rm):
        s = cp.sqrt((rm - self.rs) * (rm + 3 * self.rs))
        m = (s - rm + 3 * self.rs) / (2 * s)
        arg = cp.sqrt(2 * s / (3 * rm - 3 * self.rs + s))
        arg = cp.clip(arg, -1.0, 1.0)
        varphi = cp.arcsin(arg)

        alpha = (4 * cp.sqrt(rm / s) * ellipk(varphi, m)) % (2 * cp.pi) - cp.pi
        return alpha
    
    def mollweide(self, theta, phi):
        delta, alpha = self.spherical_to_equatorial(theta, phi)

        x = ((delta / cp.pi) * cp.cos(alpha)+1)/2 #[0,1]
        y = (cp.sin(alpha)+1)/2 #[0,1]
        return x,y
    
    def spherical_to_equatorial(self, theta, phi):
        # Coordenadas esféricas estándar
        x = cp.sin(theta) * cp.cos(phi)
        y = cp.sin(theta) * cp.sin(phi)
        z = cp.cos(theta)

        # Rotación: −π/2 alrededor del eje x
        x_rot = x
        y_rot = z
        z_rot = -y

        # Convertimos de nuevo a esféricas
        delta = cp.arcsin(z_rot)         # declinación
        alpha = ((cp.arctan2(y_rot, x_rot))-self.beta) % (2*cp.pi)   # ascensión recta

        return delta, alpha
    
    def pixel(self, phi, theta_view, alpha):
        x, y= self.mollweide(phi-alpha, theta_view)

        i = (x * self.stars_width).astype(cp.int32) % self.stars_width
        j = (y * self.stars_height).astype(cp.int32) % self.stars_height

        return i,j


    def render(self, save_path="../Multimedia/Agujero_negro/Agujero_negro_fondo_estrellado_lenteado.png"):
        # Generar malla de rayos
        i = cp.arange(self.image_size_x)
        j = cp.arange(self.image_size_y)
        ii, jj = cp.meshgrid(i, j, indexing='ij')

        x = (ii - self.image_size_x / 2) / (self.image_size_x / 2) * self.b_max_x
        y = (jj - self.image_size_y / 2) / (self.image_size_y / 2) * self.b_max_y

        b = cp.sqrt(x**2 + y**2)

        theta_view = cp.arctan2(y, x)

        # Filtrar b > rs (los que pueden llegar)
        #mask = b > self.rs
        #b_valid = cp.where(mask, b, self.rs * (1+ 1e-5))

        # Interpolar rm en GPU
        #b_vals_gpu = cp.asarray(self.b_vals_cpu)
        #rm_vals_gpu = cp.asarray(self.rm_vals_cpu)
        #rm = cp.interp(b_valid, b_vals_gpu, rm_vals_gpu)

        # Calcular deflexión
        #alpha = self.deflexion_gpu(rm)
        alpha = 0

        # Calcular dirección aparente
        phi = b / self.b_max * cp.pi / 10 - cp.pi /20

        # Convertir a píxeles
        x_pix, y_pix = self.pixel(phi, theta_view, alpha)

        # Samplear colores
        colors = self.stars_img[x_pix.get(), y_pix.get()]

        # Aplicar máscara para poner negro donde b <= rs
        #colors[~mask.get()] = [0, 0, 0]

        # Guardar imagen
        final_img = colors.reshape((self.image_size_x, self.image_size_y, 3))
        plt.imshow(final_img.astype(np.uint8))
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()

        print(f"Imagen guardada en: {save_path}")


rt = GPU_SchwarzschildRayTracer(beta=0.0, bh_mass=1.0)
rt.render()


Precomputando rm(b): 100%|██████████| 1000/1000 [00:00<00:00, 73576.54val/s]


ValueError: cannot reshape array of size 15746400 into shape (3,3240,3)

In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from scipy import optimize
from cupyx.scipy.special import ellipk


class GPU_SchwarzschildRayTracer:
    def __init__(self, beta=0.0, bh_mass=1.0):
        # Imagen de fondo (sin transponer)
        self.stars_img = np.array(Image.open("../data/Background_1.jpg").convert("RGB"))
        self.stars_height, self.stars_width = self.stars_img.shape[:2]

        self.image_size_x = self.stars_width
        self.image_size_y = self.stars_height

        # Parámetros físicos
        self.M = bh_mass
        self.rs = 2 * bh_mass

        self.r_start = 100 * self.rs
        self.r_max = self.r_start * 1.1

        self.b_max_x = 20 * self.rs / self.M
        self.b_max_y = self.b_max_x / self.image_size_x * self.image_size_y
        self.b_max = np.sqrt(self.b_max_x**2 + self.b_max_y**2)

        self.beta = beta

        # Precálculo de rm(b) en CPU
        self._precompute_rm_table()

    def _precompute_rm_table(self, N=1000):
        b_vals = np.linspace(self.rs * 1.01, self.b_max, N)
        rm_vals = []

        for b in tqdm(b_vals, desc="Precomputando rm(b)", unit="val"):
            def rmin(R): return 1 / b**2 - 1 / R**2 + self.rs / R**3
            try:
                sol = optimize.root_scalar(rmin, bracket=[self.rs * 1.01, self.r_max], method='brentq')
                rm_vals.append(sol.root)
            except:
                rm_vals.append(self.rs)

        self.b_vals_cpu = b_vals
        self.rm_vals_cpu = np.array(rm_vals)

    def deflexion_gpu(self, rm):
        s = cp.sqrt((rm - self.rs) * (rm + 3 * self.rs))
        m = (s - rm + 3 * self.rs) / (2 * s)
        arg = cp.sqrt(2 * s / (3 * rm - 3 * self.rs + s))
        arg = cp.clip(arg, -1.0, 1.0)
        varphi = cp.arcsin(arg)

        alpha = (4 * cp.sqrt(rm / s) * ellipk(varphi, m)) % (2 * cp.pi) - cp.pi
        return alpha

    def mollweide(self, theta, phi):
        delta, alpha = self.spherical_to_equatorial(theta, phi)
        x = ((delta / cp.pi) * cp.cos(alpha) + 1) / 2
        y = (cp.sin(alpha) + 1) / 2
        return x, y

    def spherical_to_equatorial(self, theta, phi):
        x = cp.sin(theta) * cp.cos(phi)
        y = cp.sin(theta) * cp.sin(phi)
        z = cp.cos(theta)

        x_rot = x
        y_rot = z
        z_rot = -y

        delta = cp.arcsin(z_rot)
        alpha = (cp.arctan2(y_rot, x_rot)) % (2 * cp.pi)
        return delta, alpha

    def pixel(self, phi, theta_view, alpha):
        x, y = self.mollweide(phi - alpha, theta_view)

        i = (x * self.stars_width).astype(cp.int32) % self.stars_width
        j = (y * self.stars_height).astype(cp.int32) % self.stars_height

        return i, j

    def render(self, save_path="../Multimedia/Agujero_negro/Agujero_negro_fondo_estrellado_lenteado.png"):
        i = cp.arange(self.image_size_x)
        j = cp.arange(self.image_size_y)
        ii, jj = cp.meshgrid(i, j, indexing='ij')

        x = (ii - self.image_size_x / 2) / (self.image_size_x / 2) * self.b_max_x
        y = (jj - self.image_size_y / 2) / (self.image_size_y / 2) * self.b_max_y

        b = cp.sqrt(x**2 + y**2)
        theta_view = cp.arctan2(y, x)

        # Límite de rayos que escapan (descomentar para activar lente gravitacional)
        # mask = b > self.rs
        # b_valid = cp.where(mask, b, self.rs * (1 + 1e-5))

        # Interpolación y cálculo de deflexión (descomentar para lente real)
        # b_vals_gpu = cp.asarray(self.b_vals_cpu)
        # rm_vals_gpu = cp.asarray(self.rm_vals_cpu)
        # rm = cp.interp(b_valid, b_vals_gpu, rm_vals_gpu)
        # alpha = self.deflexion_gpu(rm)

        alpha = 0  # sin deflexión aún
        phi = b / self.b_max * cp.pi    # sin escalado artificial

        x_pix, y_pix = self.pixel(phi, theta_view, alpha)

        # Acceder a la imagen con [y, x]
        print("stars_img shape:", self.stars_img.shape)
        print("x_pix min/max:", x_pix.get().min(), x_pix.get().max())
        print("y_pix min/max:", y_pix.get().min(), y_pix.get().max())

        colors = self.stars_img[y_pix.get(), x_pix.get()]

        final_img = colors.reshape((self.image_size_x, self.image_size_y, 3))
        plt.imshow(final_img.astype(np.uint8))
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()

        print(f"Imagen guardada en: {save_path}")


# Ejecutar
rt = GPU_SchwarzschildRayTracer(beta=0.0, bh_mass=1.0)
rt.render()


Precomputando rm(b): 100%|██████████| 1000/1000 [00:00<00:00, 38554.84val/s]


stars_img shape: (1620, 3240, 3)
x_pix min/max: 817 2422
y_pix min/max: 0 1619
Imagen guardada en: ../Multimedia/Agujero_negro_fondo_estrellado_lenteado.png
